### Middleware Basics

In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

### Summarize messages

In [2]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware

from langchain.chat_models import init_chat_model

model_gemini_flash = init_chat_model("gemini-2.5-flash", model_provider="google_genai", timeout=30, temperature=0)
model_llama_groq = init_chat_model("llama-3.1-8b-instant", model_provider="groq", timeout=30, temperature=0)

agent = create_agent(
    model=model_llama_groq,
    checkpointer=InMemorySaver(),
    middleware=[SummarizationMiddleware(
        model=model_llama_groq,
        trigger=("tokens", 100),
        keep=("messages",1)
    )]
)



In [3]:
from langchain.messages import HumanMessage, AIMessage
response = agent.invoke(
    {"messages": [
        HumanMessage(content="What is the capital of the moon?"),
        AIMessage(content="The capital of the moon is Lunapolis."),
        HumanMessage(content="What is the weather in Lunapolis?"),
        AIMessage(content="Skies are clear, with a high of 120C and a low of -100C."),
        HumanMessage(content="How many cheese miners live in Lunapolis?"),
        AIMessage(content="There are 100,000 cheese miners living in Lunapolis."),
        HumanMessage(content="Do you think the cheese miners' union will strike?"),
        AIMessage(content="Yes, because they are unhappy with the new president."),
        HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?"),
        ]},
    {"configurable": {"thread_id": "1"}}
)

In [4]:
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

Here is a summary of the conversation to date:

The moon does not have a capital. Lunapolis is a fictional city. The conversation appears to be a series of humorous and fictional exchanges about the moon and its inhabitants. 

Key points extracted:
- The moon does not have a capital.
- Lunapolis is a fictional city.
- The weather in Lunapolis is extremely hot and cold.
- There are 100,000 cheese miners living in Lunapolis.
- The cheese miners' union may strike due to dissatisfaction with the new president.
================================ Human Message =================================

If you were Lunapolis' new president how would you respond to the cheese miners' union?
================================== Ai Message ==================================

My fellow Lunapolitans, esteemed members of the Cheese Miners' Union, and honored guests from the lunar surface, I, President Zara Sparkspanner, stand bef

### Trim/delete messages

In [ ]:
from typing import Any
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent,AgentState
from langchain.messages import ToolMessage
from langgraph.checkpoint.memory import InMemorySaver

@before_agent
def trim_messages(state:AgentState, runtime:Runtime) -> dict[str,Any]|None:
    "Remove all the tool messages from the prompt"
    messages = state["messages"]
    tool_messages = [ m for m in messages if isinstance(m,ToolMessage)]
    return {"messages":[RemoveMessage(id=m.id) for m in tool_messages]}


agent = create_agent(
    model=model_llama_groq,
    checkpointer=InMemorySaver(),
    middleware=[trim_messages]
)

response = agent.invoke(
    {"messages": [
        HumanMessage(content="My device won't turn on. What should I do?"),
        ToolMessage(content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"),
        AIMessage(content="Is the device plugged in and turned on?"),
        HumanMessage(content="Yes, it's plugged in and turned on."),
        ToolMessage(content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"),
        AIMessage(content="Is the device showing any lights or indicators?"),
        HumanMessage(content="What's the temperature of the device?")
        ]},
    {"configurable": {"thread_id": "2"}}
)



TypeError: trim_messages() takes 1 positional argument but 2 were given